In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

/home/arlisscoates/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# remove unrelated columns
drop_columns = ['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'loan_status', 'url', 'desc', 
               'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
               'recoveries',  'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
                 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low','policy_code', 
                'application_type', 'disbursement_method', 'debt_settlement_flag',  'debt_settlement_flag_date',
                'settlement_status',  'settlement_date', 'settlement_amount', 'settlement_percentage', 
                'settlement_term', 'sec_app_inq_last_6mths', 'orig_projected_additional_accrued_interest', 
                'payment_plan_start_date','deferral_term', 'total_rec_int', 'zip_code', 'title', 'pymnt_plan',
               'addr_state', 'emp_title']

In [3]:
accepted = pd.read_csv('accepted_sample.csv', low_memory=False).drop(columns=drop_columns)
pd.set_option('display.max_columns', 999)
accepted.head()

KeyError: "['id' 'member_id' 'funded_amnt' 'funded_amnt_inv' 'loan_status' 'url'\n 'desc' 'initial_list_status' 'out_prncp' 'out_prncp_inv' 'total_pymnt'\n 'total_pymnt_inv' 'recoveries' 'collection_recovery_fee' 'last_pymnt_d'\n 'last_pymnt_amnt' 'next_pymnt_d' 'last_credit_pull_d'\n 'last_fico_range_high' 'last_fico_range_low' 'policy_code'\n 'application_type' 'disbursement_method' 'debt_settlement_flag'\n 'debt_settlement_flag_date' 'settlement_status' 'settlement_date'\n 'settlement_amount' 'settlement_percentage' 'settlement_term'\n 'sec_app_inq_last_6mths' 'orig_projected_additional_accrued_interest'\n 'payment_plan_start_date' 'deferral_term' 'total_rec_int' 'zip_code'\n 'title' 'pymnt_plan' 'addr_state' 'emp_title'] not found in axis"

In [ ]:
list(accepted.columns)

### Missing Values

In [ ]:
accepted.isna().mean().sort_values(ascending=False).head(65).plot(kind='bar', figsize=(12,5))

Majority of variables related to joint applications are NAN. Therefore, we create a dummy variable for joint application and remove all variables for joint application as most values are NAN.

In [ ]:
accepted['isjoint'] = 1*(~accepted['sec_app_fico_range_high'].isna())

In [ ]:
sec_columns = accepted.columns[accepted.columns.str.contains('sec_app')]
sec_columns = np.hstack((sec_columns, accepted.columns[accepted.columns.str.contains('_joint')]))
accepted.drop(columns = sec_columns, inplace=True)
sec_columns

Similarly for hardship variables. Most applicant didn't have hardship status. We create a dummy to refelct whether a person had hardship before or not and remove the hardship related columns

In [ ]:
accepted['ishardship'] = 1*(~accepted['hardship_end_date'].isna())

In [ ]:
hard_columns = accepted.columns[accepted.columns.str.contains('hardship_')]
accepted.drop(columns = hard_columns, inplace=True)
hard_columns

For the remaining variables, we replace NaN values with zero

In [ ]:
accepted = accepted.fillna(0)

### Data Type

In [ ]:
accepted.dtypes.value_counts().plot(kind='bar');

### Data Cleaning

Convert employement length from string to numeric by extracting numbers

In [ ]:
accepted['emp_length'] = accepted['emp_length'].str.extract('(\d+)')[0].fillna(0).astype(int)

Extract the year from earliest_cr_line

In [ ]:
accepted['earliest_cr_line'] = accepted['earliest_cr_line'].str.extract('(\d+)')[0].fillna(2018).astype(int)

Extract the year from issue_d

In [ ]:
accepted['issue_d'] = accepted['issue_d'].str.extract('(\d+)')[0].fillna(2018).astype(int)

### EDA

Distribution of interest rate, majority of loan have interest rate between 7% to 15%

In [ ]:
sns.kdeplot(accepted['int_rate'], shade=True)

Correlation between target and numeric columns

In [ ]:
accepted.drop(columns=['int_rate']).corrwith(accepted['int_rate']).\
            sort_values().dropna().plot(kind='bar', figsize=(15,5))
plt.ylim([-1, 1]);

Among features, Fico scores and the number of accounts opened in past 12 months have the highest correlation with the target. Fico score has negative correlation which means that higher Fico score leads to lower interest rate. On the other hand, number of account has positive correlation which means that if a person has opened many account during the last 12 months, his interest rate would be higher

This figure shows that longer loan duration leads to higher insterest rate

In [ ]:
sns.boxplot(x='term', y='int_rate', data=accepted);

Lending club uses a model to assing borrower a grade based on their profile. Grade A has the lowest risk for the investor and grade G has the highest. Below figure shows that the interest rate is highly correlated with borower grade

In [ ]:
sns.boxplot(x='grade', y='int_rate', data=accepted.sort_values('grade'));

This figure shows the correlation between loan amount and interest rate. Higher loan amounts tend to have higher interest rate

In [ ]:
sns.lmplot(x='loan_amnt', y='int_rate', data=accepted,height=5, aspect=2)

FICO score is impacting interest rate for the borower. Better FICO score (higher values) leads to lower interest rate

In [ ]:
sns.lmplot(x='fico_range_high', y='int_rate', hue='term', data=accepted,height=5, aspect=2)

In [ ]:
sns.lmplot(x='earliest_cr_line', y='int_rate', hue='term', data=accepted,height=5, aspect=2)

Interest Rate versus year, the interest rate is also a function of nationwide economy which varies over time

In [ ]:
accepted.groupby('issue_d')['int_rate'].mean().plot(style='bo-')
plt.xticks(accepted['issue_d'].unique())

The following figure shows the interest rate for different loan purpose. It shows that certain loan purpose has lower interest rate such as buying a car due to lower risk to the investors

In [ ]:
plt.figure(figsize=(14,5))
sns.boxplot(x='purpose', y='int_rate', data=accepted);
plt.xticks(rotation=90);

In the following figure the distirbution of interest rate for different home ownership status is depicted. Borrowers who are renting tend to have higher interest rate on their loan application. This is due to the fact that home owners (or people with mortgage) carry less task to the investors, as their home can be later used as a collateral to repay the loan

In [ ]:
for item in accepted['home_ownership'].unique():
    sns.kdeplot(accepted[accepted['home_ownership'] == item]['int_rate'], label = item, shade=True)
plt.xlabel('Interest Rate')
plt.ylabel('Distribution')

## Modeling

In [ ]:
accepted = accepted.drop(columns=['grade', 'sub_grade', 'installment'])

In [ ]:
accepted_dummy = pd.get_dummies(accepted)
X_train = accepted_dummy.drop(columns='int_rate')

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X_train)
y = accepted_dummy['int_rate'].values
print(X.shape)

### Outlier Detection Using Linear Regression

We fit a linear regression model on the selected features and then caluclate the residual for each observation on the dataset. The residuals versus fitted values are plotted and outliers are idenitfied. Below figure shows that there is one outlier in our dataset. It also shows that there is a non-linear trend in the residuals which is not captured by the linear model. Therefore, we expect that the linear model does not perform well in this problem

In [ ]:
lin = LinearRegression()
lin.fit(X, y);
fitted = lin.predict(X)
res = y - fitted
outlier = abs(res) > 20
plt.plot(fitted, res, 'ko', alpha = 0.1)
plt.plot(fitted[outlier], res[outlier], 'ro')
plt.xlabel('Fitted Value')
plt.ylabel('Residual');

Outliers are removed from the dataset

In [ ]:
X = X[~outlier,:]
y = y[~outlier]

### Linear Regression Feature Selection

We use stepwise feature selection with cross-validation and a linear regression model

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=123)
lin = LinearRegression()
rfecv = RFECV(estimator=lin, step=1, cv=kf, scoring='neg_mean_squared_error', min_features_to_select= 5)
rfecv.fit(X, y)

In [ ]:
selected = rfecv.get_support()
indx = np.argmin(-rfecv.grid_scores_)

plt.plot(range(1, len(rfecv.grid_scores_) + 1), -rfecv.grid_scores_, 'bo-')
plt.plot(range(1, len(rfecv.grid_scores_) + 1)[indx], -rfecv.grid_scores_[indx], 'ro')
plt.yscale('log')
plt.xlabel('Number of Features')
plt.ylabel('RMSE');

Stepwise feature selection has selected 70 features with the lowest RMSE

In [ ]:
selected.sum()

### Lasso Feature Selection

In [ ]:
lasso_cv = LassoCV(cv=10, random_state=123, max_iter = 400, tol =1e-2)
lasso_cv.fit(X,y);

In [ ]:
lasso_mse = lasso_cv.mse_path_.mean(axis=1)
plt.plot(lasso_cv.alphas_, lasso_mse, 'b-')
plt.plot(lasso_cv.alphas_[lasso_mse.argmin()], lasso_mse[lasso_mse.argmin()], 'ro')
plt.xscale('log')
plt.xlabel('Alpha')
plt.ylabel('MSE');
plt.title('Optimal Lasso: {0:.3f}'.format(lasso_cv.alphas_[lasso_mse.argmin()]));

In [ ]:
lasso = Lasso(alpha = lasso_cv.alphas_[lasso_mse.argmin()])
lasso.fit(X,y);
selected_lasso = np.abs(lasso.coef_) > 0.1
lasso_df = pd.Series(lasso.coef_[selected_lasso], index=X_train.columns[selected_lasso])
lasso_df.sort_values().plot(kind='bar', figsize=(15,5));

### Random Forest Feature Importance

In [ ]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X,y)
imp = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
imp.head(50).plot(kind='bar', figsize=(15,5));

### Cross Validation and Model Selection

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)

In [ ]:
gb = GradientBoostingRegressor(n_estimators=100, max_depth=8)
gb_scores = cross_val_score(gb, X, y, cv=kf, scoring='r2')

In [ ]:
rf = RandomForestRegressor(n_estimators=100)
rf_scores = cross_val_score(rf, X, y, cv=kf, scoring='r2')

In [ ]:
lin = LinearRegression()
lin_scores = cross_val_score(lin, X[:,selected], y, cv=kf, scoring='r2')

In [ ]:
svr = SVR(gamma='auto')
svr_scores = cross_val_score(svr, X[:,selected], y, cv=kf, scoring='r2')

In [ ]:
tree = DecisionTreeRegressor()
tree_scores = cross_val_score(tree, X, y, cv=kf, scoring='r2')

In [ ]:
lasso = Lasso(alpha = lasso_cv.alphas_[lasso_mse.argmin()])
lasso_scores = cross_val_score(tree, X, y, cv=kf, scoring='r2')

In [ ]:
res = pd.DataFrame({'gb': gb_scores, 'rf': rf_scores, 'lin': lin_scores, 'svr': svr_scores, 
                    'tree': tree_scores, 'lasso': lasso_scores})
res

In [ ]:
sns.barplot(x="model", y="r2", data=res.melt(var_name='model', value_name='r2'));

### Model Optimization

From above cross-validation, we can se Gradient Boosting has the highest R2 value and outperforms other model. In the following section, we use grid search CV method to optimize the hyperparamteres of the Gradient Boosting model and improve its performance

In [ ]:
params = {'n_estimators': [100, 150, 200],
          'max_depth': [4, 6, 8],
          'max_features': ['sqrt', 'log2']}
gb = GradientBoostingRegressor(random_state = 123)
grid = GridSearchCV(gb, params, cv=3, n_jobs=4, scoring='r2')
grid.fit(X,y);
grid.best_estimator_

In [ ]:
gb_scores = cross_val_score(grid.best_estimator_, X, y, cv=kf, scoring='r2')
print('Best Model R2: {0:.2f} +- {1:.2f}'.format(gb_scores.mean(), gb_scores.std()))

The GB with 200 estimators, max_depth of 4, max_feature of 'sqrt' has the optimal performance. The cross-validation shows the R2 of 0.65 which is better than our plain GB